In [1]:
# Import general libraries
import json
import pandas as pd

# Import local functions
from general_functions import awsOps

In [2]:
# Raw RegHub categories with minor cleaning
Legal_documents = ["regulation","regulations","order","Commission Implementing Decision",
                  "act","comission decision","amendment","directive","rule","rts",
                  "technical specification","Council Framework Decision","richtlinie",
                  "verordnung","regulatory technical standards","regulatory specification",
                  "legislation","gesetz"]

Warnings_Enforcements_Sanctions = ["Fine","accuse","fined","fines","Milliardenbusse",
                    "lawsuite","Strafe","Warnung","Sanction","Sanctions","Sanktions",
                    "schadenersatzanspruch","schadenersatzansprüche","schadenersatz verurteilt",
                    "Sanktionen","richterliche entscheidung","juristische entscheidung",
                    "gerichtliche","is not authorised to","administrative fine",
                    "disciplinary fine","unlawful use of","has not granted",
                    "launches investigation","Sanktion","Bafin investigates",
                    "has not been granted authorisation","without our authorisation",
                    "keine Erlaubnis nach dem","unerbetene Anrufe","prohibits unauthorised",
                    "unauthorised provision","unterliegt nicht der Aufsicht","withdrawal",
                    "unlicenced entity","fines","breaches","verstoß","verstöße",
                    "suspends","verstösse","Strafe","unauthorised business","ermittelt gegen",
                    "Strafmaßnahme","suspension of trading","Warning","untersagt die unerlaubt",
                    "Sanction","Enforcement","Achtung!","Vollstreckung","Sanktionsmaßnahme",
                    "Durchsetzung","warnt Anleger","Betroffene","mögliche Nachteile",
                    "verhängt Sanktion","Kundmachung","Öffentlichkeit informieren"]

Science_papers = ["issues alert","whitepaper","working paper","white paper","wissenschaftlicher artikel",
                 "wissenschaftliche publikation","studie","study","wissenschaftlicher aufsatz",
                 "essay","studie:","wissenschaftliche publikation","science paper","wissenschaftliche Abhandlung",
                 "wissenschaftliche Arbeit","publikation"]

Reports = ["released a report","report suggests","report","bericht","survey","umfrage","poll"]

Official_statements = ["letter","statement","speech","opinion","kapitalmarktinformation","stellungnahme",
                    "ansprache","kundmachung","Bekanntmachung","offizielle Mitteilung","Verkundbarung",
                    "Verlautbarung","eröffnungsrede","bundestagsrede"]

Guidelines = ["guideline","rundschreiben","circular","faq","q&a","questions & answers","questions and answers",
             "leitfaden","anleitung","hinweis","handlungsanweisung","norm","häufig gestellte fragen",
             "Hilfestellung","Anleitung","leitlinie"]

Press_releases = ["press release","newsletter","pressemitteilung","pressemeldung","pressebericht",
                "press announcement","press statement","media release","news release","press communiqué"]

Personnel_changes = ["Neuer Chef","Neuer Head of","übernimmt Leitung","zieht in Vorstand",
                   "zieht in Aufsichtsrat","mit neuem Bereichs","leitet Generationenwechsel",
                   "stockt Personal","ernennt C","ins Führungsteam","holt renommierte Manager",
                   "besetzt Spitzenposition","Neuer Chief","hiring","job candidate","indeed.com",
                   "new job","job role","trainee","talent scout","talent acquisition","recruitment",
                   "head hunter","executive search","new staff","Professionals search","job search",
                   "talent identification","job description","interviewee","job interview",
                   "job application","employment contract","recruitment process","external recruitment",
                   "candidate attraction","job profile","candidate profile","candidate skills",
                   "required skills","Recruiting","Job","Arbeitsplatzrolle","Praktikantin",
                   "Talentsucher","Talentakquise","Rekrutierung","Headhunter","executive search",
                   "neue Mitarbeiter","recruiter","bewerbermesse","jobmesse","Stellenbeschreibung",
                   "Dienstvertrag","Arbeitsvertrag","Rekrutierungsprozess","externe Rekrutierung",
                   "Kandidatengewinnung","Stellenangebot","Stellenanzeige","Stellenausschreibung",
                   "Job Vakanz","Job Posting","Job Postings","(mwd)","(mfd)","(fmd)","Praktikant:in",
                   "Assitent:in","(mwdivers)"]

Market_Analysis = ["Aktien Europa","behält die Bewertung","Kurserholung","Stock:","'halten'","'kaufen'",
                  "Märkte am Morgen","DAX","Dow Jones","chartanalyse","gross profit margin","ISIN",
                  "stock rating","'verkaufen'","'hold'","'buy'","'sell'","'moderate buy'","price prediction",
                  "price remains above","price remains below","best stocks","raised its price","Price Target",
                  "Release of capital market information","Analysts have given","'Buy'","'Moderate Buy'",
                  "'Sell'","'Hold'","reiteratives its rating on","increase in short interest","buy rating",
                  "sell rating","hold rating","aktienrückkauf","kurssturz","drückt Kurse","Aktienkurs",
                  "fall in the shares","worth of shares","Milliardenplatzierung","roundupaktien","estimatekurse",
                  "realtime kurse","aktie angepasst","Aktie:","aktie stabil","aktie volatil","der kurs der aktie",
                  "aktie news","shareholder","aktie dämmt","der aktionaer","Aktie News","Kursplus","Kursminus",
                  "Kurssprung","der aktionär","'buy'","'sell'","boerse.de","aktiencheck","Dividendensaison",
                  "DAXAktien","DAX-Aktien","share price","Aktie bü0t","Aktie gewinnt","stock loss","stock win",
                  "kursziel","prognostizierten die Analysten","kursgewinn","aktien im sinkflug","aktie im",
                  "Preisrally","Price Update","Bearish Signal","Bullish Signal","bear market","bull market",
                  "bear signal","bull signal","Price Prediction:","losses below $","losses below \u20AC",
                  "recover to $","recover to \u20AC","consolidates above $","resistance at $","reclaims $",
                  "to break $","drop to $","drop to \u20AC","price correction","Preiskorrektur","bounces above $",
                  "bounces above \u20AC","rejection at $","rejection at \u20AC","pauses above $","pauses below $",
                  "revisit $","slumps above $"]

In [3]:
# We worked with the RegHub categories in Python
# and stored the results in seperate JSON files.
# The codeblocks with the modified keywords
# have been deleted for better readability.

# Dictionary to store the category lists
cat_data = {
    "Legal_documents": Legal_documents,
    "Warnings_Enforcements_Sanctions": Warnings_Enforcements_Sanctions,
    "Science_papers": Science_papers,
    "Reports": Reports,
    "Official_statements": Official_statements,
    "Guidelines": Guidelines,
    "Press_releases": Press_releases,
    "Personnel_changes": Personnel_changes,
    "Market_Analysis": Market_Analysis
}

In [4]:
# Specify the file path where you want to save the JSON file
json_file_path = "../../categories_raw.json"

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(cat_data, json_file)

In [5]:
# Specify the file path where the JSON file is stored
json_file_path = "../../categories_v2.json"

# Read the data from the JSON file
with open(json_file_path, 'r') as json_file:
    category_dict = json.load(json_file)

In [6]:
# Transform the data to bring it into csv-format
df = pd.DataFrame()

for key, value in category_dict.items():
    df[key] = pd.Series(value)

In [7]:
# Save the data as df at desired location
# Local
df.to_csv("../../rule_labels_v1.csv")

# AWS
# Access aws credentials from json file
with open("../aws_credentials.json", 'r') as file:
    aws_creds_json = json.load(file)
# Specify s3 bucket
bucket = "fs-reghub-news-analysis"

# Connect to aws and upload the file
aws = awsOps(aws_creds_json)
aws.upload_file(bucket=bucket, path="../../rule_labels_v1.csv", name="rule_labels_v1.csv")

'File uploaded'